In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

# chargement des donnees + calcul baseline

data_folder = Path('./')

df = pd.read_csv(data_folder/'dataset.csv', dtype='int8')
y = pd.read_csv(data_folder/'y.csv', dtype='int8')

print(df.shape, y.shape)

# baseline
# print(y['0'].value_counts()[1])
# 1 - (y['0'].value_counts()[1] / y['0'].value_counts()[0])

(1986, 37) (1986, 1)


In [42]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for train_index, test_index in cv.split(df, y):
    
    poly = PolynomialFeatures(degree=3)
    X_ = poly.fit_transform(df.iloc[train_index])
    predict_ = poly.fit_transform(df.iloc[test_index])

    clf = linear_model.LinearRegression()
#     reg = clf.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
    reg = clf.fit(X_, y.iloc[train_index].values.ravel())
#     print(y.iloc[test_index].values.ravel())
#     print(reg.predict(predict_))
#     print(reg.predict(df.iloc[test_index]))
#     print(reg.score(df.iloc[test_index], y.iloc[test_index].values.ravel()))
#     print(mean_squared_error(y.iloc[test_index].values.ravel(), reg.predict(df.iloc[test_index])))

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
regr = RandomForestRegressor(max_depth=9, random_state=42, n_estimators=400)

for train_index, test_index in cv.split(df, y):
    regr.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
#     print(regr.score(df.iloc[test_index], y.iloc[test_index].values.ravel()))
    print(mean_squared_error(y.iloc[test_index].values.ravel(), regr.predict(df.iloc[test_index])))

C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Paul\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


1.6601962803954202
1.6644622583702986
1.416033569663452


In [17]:
import xgboost as xgb
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# training du xgboost

#param_range = [0.001,0.01,0.1,1,10,100]
param_range = [1]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for i in param_range:
    classifier = xgb.XGBClassifier(max_depth=7, n_estimators=500, learning_rate=0.01, silent=0, random_state=42)

    acc = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        y_hat = model.predict(df.iloc[test_index])
        # Making the Confusion Matrix
        cm = confusion_matrix(y.iloc[test_index], (y_hat>0.5))
        print('The Confusion Matrix is: ','\n', cm)
        # Calculate the accuracy on test set
        predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
        print('The Accuracy on Test Set is: ', predict_accuracy_on_test_set)
        
#         aucs.append(roc_auc)

#     mean_tpr = np.mean(tprs, axis=0)
#     mean_tpr[-1] = 1.0
#     mean_auc = auc(mean_fpr, mean_tpr)
    


    print('param=%s Accuracy %0.3f' % (str(i), predict_accuracy_on_test_set))
#     print(aucs)

# The Confusion Matrix is:  
#  [[488  30]
#  [ 92  52]]
# The Accuracy on Test Set is:  0.8157099697885196

# The Confusion Matrix is:  
#  [[581   9]
#  [ 53  19]]
# The Accuracy on Test Set is:  0.9063444108761329

The Confusion Matrix is:  
 [[477  41]
 [ 88  57]]
The Accuracy on Test Set is:  0.8054298642533937
The Confusion Matrix is:  
 [[488  30]
 [ 92  52]]
The Accuracy on Test Set is:  0.8157099697885196
The Confusion Matrix is:  
 [[468  49]
 [ 86  58]]
The Accuracy on Test Set is:  0.7957639939485628
param=1 Accuracy 0.796


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp

#Mean ROC (AUC = 0.844) train_concat_90_00001
#param_range = [0.001,0.01,0.1,1,10,100]
#param_range = [0.001]
param_range = [0.08]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for i in param_range:
    aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    classifier = LogisticRegression(C=i, random_state=42)
    #LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        if hasattr(model, "predict_proba"):
            probas_ = model.predict_proba(df.iloc[test_index])[:, 1]
        else:  # use decision function
            probas_ = model.decision_function(df.iloc[test_index])
        fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    print('param=%s Mean ROC (AUC = %0.3f)' % (str(i), mean_auc))
    print(aucs)

param=0.08 Mean ROC (AUC = 0.798)
[0.7982891758753828, 0.8102209352209353, 0.7854072641306684]
